In [4]:
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset
from collections import Counter
import pickle as pkl
import random
import pdb
import pandas as pd
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
import string

PAD_IDX = 0
UNK_IDX = 1

train = 'data/snli_train.tsv'
val = 'data/snli_val.tsv'

In [5]:
def load_emb_matrix():
    #load fasttext word vectors
    words_to_load = 50000

    with open('wiki-news-300d-1M-subword.vec') as f:
        #remove the first line
        firstLine = f.readline()
        loaded_embeddings = np.zeros((words_to_load + 2, 300))
        words2id = {}
        idx2words = {}
        #ordered_words = []
        for i, line in enumerate(f):
            if i >= words_to_load: 
                break
            s = line.split()
            loaded_embeddings[i + 2 , :] = np.asarray(s[1:])
            words2id['<pad>'] = PAD_IDX
            words2id['<unk>'] = UNK_IDX
            words2id[s[0]] = i + 2
            idx2words[0] = '<pad>'
            idx2words[1] = '<unk>'
            idx2words[i + 2] = s[0]
   

    return words2id,idx2words,loaded_embeddings

In [6]:
words2id,idx2words,loaded_embeddings = load_emb_matrix()

pkl.dump(words2id, open(f'data/words2id.pkl', 'wb'))
pkl.dump(idx2words, open(f'data/idx2words.pkl', 'wb'))
pkl.dump(loaded_embeddings, open(f'data/embedding_matrix.pkl', 'wb'))

In [ ]:
# Load English tokenizer, tagger, parser, NER and word vectors
tokenizer = spacy.load('en_core_web_sm')
punctuations = string.punctuation

# lowercase and remove punctuation
def tokenize(sent):
    tokens = tokenizer(sent)
    return [token.text.lower() for token in tokens 
            if (token.text not in punctuations) & (token.text not in STOP_WORDS)]

In [ ]:
def tokenize_dataset(dataset):
    token_dataset = []
    #all_tokens = []
    for sample in dataset:
        tokens = tokenize(sample)
        token_dataset.append(tokens)
        #all_tokens += tokens

    return token_dataset

In [ ]:
train_tokens_1 = tokenize_dataset(train_data['sentence1'])
train_tokens_2 = tokenize_dataset(train_data['sentence2'])
pkl.dump(train_tokens_1, open("data/train_data_tokens_1.p", "wb"))
pkl.dump(train_tokens_2, open("data/train_data_tokens_2.p", "wb"))

val_tokens_1 = tokenize_dataset(val_data['sentence1'])
val_tokens_2 = tokenize_dataset(val_data['sentence2'])
pkl.dump(val_tokens_1, open("data/val_data_tokens_1.p", "wb"))
pkl.dump(val_tokens_2, open("data/val_data_tokens_2.p", "wb"))